In [ ]:
pip install matplotlib==3.5.1

In [ ]:
pip install ipywidgets as widgets

In [ ]:
pip install torch==1.10.1

In [ ]:
pip install torchvision==0.11.2

In [ ]:
pip install torchaudio==0.10.1

In [2]:
pip list

Package            VersionNote: you may need to restart the kernel to use updated packages.

------------------ -------
asttokens          2.0.8
backcall           0.2.0
colorama           0.4.5
cycler             0.11.0
debugpy            1.6.3
decorator          5.1.1
entrypoints        0.4
executing          1.0.0
fonttools          4.37.2
ipykernel          6.15.3
ipython            8.5.0
ipywidgets         8.0.2
jedi               0.18.1
jupyter_client     7.3.5
jupyter-core       4.11.1
jupyterlab-widgets 3.0.3
kiwisolver         1.4.4
matplotlib         3.5.1
matplotlib-inline  0.1.6
nest-asyncio       1.5.5
numpy              1.23.3
packaging          21.3
parso              0.8.3
pickleshare        0.7.5
Pillow             9.2.0
pip                22.2.2
prompt-toolkit     3.0.31
psutil             5.9.2
pure-eval          0.2.2
Pygments           2.13.0
pyparsing          3.0.9
python-dateutil    2.8.2
pywin32            304
pyzmq              24.0.0
setuptools         65.3.0

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
#import tensorflow as tf
#import lime
#import time
#import shutil
#import random
#import csv
import os,sys
#from skimage.io import imreadcd 
#from skimage.segmentation import mark_boundaries
%load_ext autoreload
%autoreload 2

#main_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir + os.sep + os.pardir)
main_dir = os.getcwd()
print(main_dir)

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Video
import os

#To get the current directory and set the path of the folder to the data folder
def getDataFolderVideoContent():
    current_dir = str(os.getcwd())
    data_folder = current_dir + "\\Datasets\\Video"
    data_folder_content = []

    #Loop through to get all the contents inside the data folder
    with os.scandir(data_folder) as entries:
        for entry in entries:
            data_folder_content.append(entry.name)
    
    return data_folder_content

#Dropdown to display all videos in data folder
video_dropdown = widgets.Dropdown(
    options = getDataFolderVideoContent(),
    description = 'Videos:',
    disabled = False,
)

display(video_dropdown)

In [ ]:
#Set the selected video to the iPyWidget video function and display the video playback
selected_video = Video.from_file('Datasets/Video/' + video_dropdown.value)
selected_video